In [21]:
import pandas as pd
import numpy as np
import datetime
from urllib.error import HTTPError
import pickle

import sys
sys.path.append('/usr/local/lib/python2.7/site-packages/')
sys.path.append('/usr/local/lib/python3.7/site-packages')


In [8]:
# using this package https://symfit.readthedocs.io/en/latest/fitting_types.html#ode-fitting
from symfit import Parameter, variables, Fit, D, ODEModel

In [9]:
# importing csv directly from website

# See this post on how to import csv directly from a website
# https://projectosyo.wixsite.com/datadoubleconfirm/single-post/2019/04/15/Reading-csv-data-from-Github---Python



In [10]:
def get_date(date_str):
# function for rewriting date & time in a uniform format.

  # date_str: (str) representing date and time
  # ex) '1/31/2020 23:59'	
  # ex) '2020-02-23T11:33:03'

  formats = ["%Y-%m-%dT%H:%M:%S", "%m/%d/%Y %H:%M", "%m/%d/%y %H:%M"]
  
  # check whether the given "date_str" matches one of the formats
  for format in formats:
    try:
       corrected_date = datetime.datetime.strptime(date_str, format).date()
       return corrected_date.isoformat()
    # if above try gives a "ValueError" move on to the next format
    except ValueError:
      continue

    # if "date_str" doesn't match any of the formats
  raise ValueError(date_str)


In [11]:
# COVID-19 data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data
# get data starting from 01/22/2020 until most recent data available

url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"
day = datetime.datetime.strptime("01-22-2020", "%m-%d-%Y").date()

data = {}
error = False 

while error == False:
    url_day = url + day.strftime("%m-%d-%Y") + '.csv'
    try:
        df = pd.read_csv(url_day)
        
        
        # get rid of unnecessary columns (Latitutde, Longitude) if such columns exist
        df.drop(columns = ["Latitude", "Longitude"], errors = "ignore", inplace = True)

        # make the date/time format of "Last Update" uniform
        df["Last Update"] = df["Last Update"].apply(get_date)

        # create a column with report date, as given by date
        # Note: Sometimes, we have duplicate data
        # for example, if King County reported their results in 03/08 and then on 03/10, then the report on 03/09 will contain 
        # information from King County on 03/08
        df["report date"] = [day.strftime("%m-%d-%Y")] * df.shape[0]

        # Make column names clearer
        # "Confirmed":  CUMULATIVE number of confirmed cases upto that date.
        # In particular, it includes people who were once confirmed in the past that are
        # either dead or recovered. 
        df.rename(columns={'Confirmed': 'cumulative_infected'}, inplace = True)

        # deaths & recovered -> recovered
        df["death_or_recovered"] = df["Deaths"] + df["Recovered"]

        # adjust the number of confirmed cases by subtracting death and recovered
        df["infected"] = df["cumulative_infected"] - df["death_or_recovered"]

        data[day.strftime("%m-%d-%Y")] = df
        
        # update to next day
        day = day + datetime.timedelta(days = 1)
    except HTTPError:
        error = True 


In [12]:
f = open("COVID_data_03_21.pkl","wb")
pickle.dump(data,f)
f.close()

In [13]:
data = pickle.load( open( "COVID_data_03_21.pkl", "rb" ) )

In [14]:
def state_data(data, state):
  # select data for selected state
  """
  --- input ---
  data: (dic) 
  state: (str) state abbreviation
  """
  
  state_name = abbreviation[state]
  
  keys = list(data.keys())
  keys.sort()
  last = keys[-1]
  columns = data[last].columns

  state_df = pd.DataFrame(columns = columns)  

  for date in data.keys():
    df = data[date]
    state_data = df[(df['Country/Region'] == 'US') & 
              ((df['Province/State'] ==  state_name) | (df['Province/State'].str.contains(state))
              )]
    state_df = state_df.append(state_data, ignore_index = True)

  # sort data, from oldest (top) to new (bottom) 
  state_df = state_df.sort_values(by = ["report date"])
  
  return state_df

In [15]:
def train_model(first_day, last_day, N, infected, death_or_recovered, n_days, beta_0 = 0.3, gamma_0 = 0.01):
    """
    --- input ---
    first_day: (datetime) first day of data
    last_day: (datetime) last day of data.
               NOTE: only data from "first_day" to "last_day" will be used in the parameter tuning process
    N: (int) number of susceptible population 
    infected: (series) data of infected people by date
                  infected[date] is the number of infected people on that date.
                ex) infected["03-02-2020"] is the number of infected people on March 2, 2020
    death_or_recovered: (series) data of number of people that died or recovered.
                  death_or_reovered[date] is the number of people that died / recovered up to that date.
    n_days: (int) number of days to make predictions. 
                  day 0 coincides with the "first_day"
    
    --- output ---
    prediction: (dict) of prediction data
    """

    # get data in correct format
    data_duration = (last_day - first_day).days +1
    data_time = np.array(range(0,data_duration)).astype(int) # time 0 corresponds to the first day appearing in dataframe
    data_infected = np.array(infected[first_day.strftime("%m-%d-%Y"):last_day.strftime("%m-%d-%Y")]).astype(int) # number of people infected 
    data_recovered = np.array(death_or_recovered[first_day.strftime("%m-%d-%Y"):last_day.strftime("%m-%d-%Y")]).astype(int)

    # define variables and parameter
    S, I, R, t = variables('S, I, R, t')
    beta = Parameter('beta', 0.3)
    gamma = Parameter('gamma', 0.01)

    # define ODE equations 
    model_dict = {
        D(S, t): - beta * S * I /N,
        D(I, t): beta * S* I / N - gamma * I,
        D(R, t): gamma * I 
    }

    # initial condition: from row 0 of dataframe
    I0 = infected[first_day.strftime("%m-%d-%Y")]
    S0 = N - I0
    R0 = death_or_recovered[first_day.strftime("%m-%d-%Y")]

    # define the model
    model = ODEModel(model_dict, initial = { t : 0, S: S0, I: I0, R: R0})

    # fit model parameters
    fit = Fit(model, t = data_time, I = data_infected, S = None, R = data_recovered)
    fit_result = fit.execute()

    # get predictions
    tvec = np.linspace(0, n_days-1, n_days)
    #tvec = np.array(0, n_days)
    outcome = model(t=tvec, **fit_result.params)
    I_pred = outcome.I
    S_pred = outcome.S
    R_pred = outcome.R 

    # save output
    prediction = {}
    prediction["first_day"] = first_day
    prediction["last_day"] = last_day
    prediction["num_days"] = n_days
    prediction["susceptible"] = N
    prediction["infected_pred"] = I_pred
    prediction["susceptible_pred"] = S_pred
    prediction["dead_or_recovered_pred"] = R_pred
    prediction["parameters"] = fit_result.params
  
    return prediction

In [16]:
def load_demographic():
    # load demograpahic data 
    dem_data = 'PEP_2018_PEPSYASEX/PEP_2018_PEPSYASEX_with_ann.csv'
    US_dem = pd.read_csv(dem_data)
    
    # select columns that has the most recent (2018) population estimate of both sex
    col = US_dem.columns[US_dem.columns.str.contains('2018', case = False ) 
                        & US_dem.columns.str.contains('sex0', case = False )]
    
    # add state info
    col = col.insert(0,"GEO.display-label")
    US_dem = US_dem[col]
    
    US_dem = US_dem.drop([0])
    
    # rename the columns into something simpler
    new_col = {x:re.sub('est72018sex0_', '',x) for x in col}
    new_col["GEO.display-label"] = "GEO"
    
    # change the names of the column 
    US_dem.rename(columns = new_col, inplace = True)
    
    # change type so that we can sum  values
    US_dem.loc[:,US_dem.columns != 'GEO'] = US_dem.loc[:,US_dem.columns != 'GEO'].astype(float)
    
    # Group ages according to 
    # https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm?s_cid=mm6912e2_w#T1_down
    
    # age 0-19
    sum_col = ["age" + str(i) for i in range(0,20)]
    US_dem["age0-19"] = US_dem[sum_col].sum( axis = 1)
      
    # 20 -44
    sum_col = ["age" + str(i) for i in range(20,45)]
    US_dem["age20-44"] = US_dem[sum_col].sum( axis = 1)
    
    # 45 - 54
    sum_col = ["age"+str(i) for i in range(45,55)]
    US_dem["age45-54"] = US_dem[sum_col].sum( axis = 1)
    
    # 55 - 64
    sum_col = ["age"+str(i) for i in range(55,65)]
    US_dem["age55-64"] = US_dem[sum_col].sum( axis = 1)
    
    # 65 - 74
    sum_col = ["age"+str(i) for i in range(65,75)]
    US_dem["age65-74"] = US_dem[sum_col].sum( axis = 1)
    
    # 75 - 84
    sum_col = ["age"+str(i) for i in range(75,85)]
    US_dem["age75-84"] = US_dem[sum_col].sum( axis = 1)
    
    # 85 <= already exists
    
    
    US_dem = US_dem[['GEO', 'age999','age0-19','age20-44','age45-54','age55-64','age65-74','age75-84', 'age85plus']]
    
    # rewrite in terms of percentage 
    
    US_dem["per0-19"] = US_dem["age0-19"] / US_dem["age999"]
    US_dem["per20-44"] = US_dem["age20-44"] / US_dem["age999"]
    US_dem["per45-54"] = US_dem["age45-54"] / US_dem["age999"]
    US_dem["per55-64"] = US_dem["age55-64"] / US_dem["age999"]
    US_dem["per65-74"] = US_dem["age65-74"] / US_dem["age999"]
    US_dem["per75-84"] = US_dem["age75-84"] / US_dem["age999"]
    US_dem["per85plus"] = US_dem["age85plus"] / US_dem["age999"]

    return US_dem

In [17]:
abbreviation = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}


In [19]:
state = "WA"
state_df = state_data(data, state)

In [24]:
# get state info
state_df = state_data(data, state)

# sum up recordings from the same date since we don't care about county level info 

infected =  state_df.groupby(['report date'])['infected'].sum()
death_or_recovered = state_df.groupby(['report date'])['death_or_recovered'].sum()

#if st.checkbox("Show dataframe"):
#    st.write(state_df)
    
# Select days     

first_report_date = "02-29-2020"
first_day = datetime.datetime.strptime(first_report_date, '%m-%d-%Y')

last_report_date = "03-18-2020"
last_day = datetime.datetime.strptime(last_report_date, "%m-%d-%Y")

P = train_model(first_day, last_day, 70000, infected, death_or_recovered, 100)






In [25]:
P

{'first_day': datetime.datetime(2020, 2, 29, 0, 0),
 'last_day': datetime.datetime(2020, 3, 18, 0, 0),
 'num_days': 100,
 'susceptible': 70000,
 'infected_pred': array([5.00000000e+00, 6.80559066e+00, 9.26311564e+00, 1.26078822e+01,
        1.71600580e+01, 2.33552146e+01, 3.17858128e+01, 4.32575075e+01,
        5.88654802e+01, 8.00977797e+01, 1.08974943e+02, 1.48238127e+02,
        2.01601646e+02, 2.74090227e+02, 3.72486156e+02, 5.05916175e+02,
        6.86610851e+02, 9.30866985e+02, 1.26023046e+03, 1.70288235e+03,
        2.29513877e+03, 3.08284113e+03, 4.12219307e+03, 5.47927894e+03,
        7.22711467e+03, 9.43879250e+03, 1.21754405e+04, 1.54688697e+04,
        1.93013623e+04, 2.35886691e+04, 2.81749226e+04, 3.28466734e+04,
        3.73660277e+04, 4.15130615e+04, 4.51222595e+04, 4.81011047e+04,
        5.04283956e+04, 5.21383494e+04, 5.32995301e+04, 5.39956838e+04,
        5.43118484e+04, 5.43260845e+04, 5.41056595e+04, 5.37061492e+04,
        5.31721651e+04, 5.25388089e+04, 5.18333